# Buat Nama Kelompok DKK

## Import

In [2]:
#Ini gw copas dari tucil dulu ya sementara, nanti yang ga butuh apus ae

import numpy as np 
import random
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import itertools

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree

from sklearn import svm, datasets
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier




%matplotlib inline

## Read data

In [3]:
heartTrain = pd.read_csv('tubes2_HeartDisease_train.csv')
heartTest = pd.read_csv('tubes2_HeartDisease_test.csv')
heartTrain['Column4'].unique()

array(['125', '158', '135', '120', '130', '150', '115', '118', '142',
       '132', '?', '160', '110', '112', '155', '180', '108', '104', '140',
       '100', '200', '95', '170', '122', '126', '138', '124', '144',
       '145', '101', '128', '105', '92', '114', '80', '116', '117', '152',
       '178', '154', '146', '94', '136', '148', '156', '106', '98', '134',
       '192', '172', '190', '165', '174', '185', '96', '127', '102', '0',
       '129'], dtype=object)

In [13]:
# Convert string to numeric, convert ? to NAN
heartTrain = heartTrain.apply(pd.to_numeric,errors = 'coerce')

# NaN count
heartTrain.isna().sum()

Column1       0
Column2       0
Column3       0
Column4      47
Column5      24
Column6      78
Column7       2
Column8      44
Column9      44
Column10     49
Column11    262
Column12    514
Column13    408
Column14      0
dtype: int64